In [1]:
import topmost
from topmost.data import download_dataset

device = "cuda" # or "cpu"
dataset_dir = "./datasets/20NG"
download_dataset('20NG', cache_path='./datasets')

/home/xiaobao/local/miniconda3/envs/topmost/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


https://raw.githubusercontent.com/BobXWu/TopMost/master/data/20NG.zip


100%|██████████| 11927380/11927380 [00:00<00:00, 12169265.06it/s]


In [2]:
########################### Neural Topic Models ####################################
# dataset for neural topic models.
# For combinedTM, add contextual_embed=True.
dataset = topmost.data.BasicDataset(dataset_dir, read_labels=True, device=device)

train_size:  11314
test_size:  7532
vocab_size:  5000
average length: 110.543


In [7]:
# create a model
# model = topmost.models.ProdLDA(dataset.vocab_size)
model = topmost.models.ETM(dataset.vocab_size, pretrained_WE=dataset.pretrained_WE)
# model = topmost.models.DecTM(dataset.vocab_size)
# model = topmost.models.TSCTM(dataset.vocab_size)
# model = topmost.models.CombinedTM(dataset.vocab_size, dataset.contextual_embed_size)
# model = topmost.models.NSTM(dataset.vocab_size, pretrained_WE=dataset.pretrained_WE)
# model = topmost.models.ECRTM(dataset.vocab_size, pretrained_WE=dataset.pretrained_WE)
model = model.to(device)

# create a trainer
trainer = topmost.trainers.BasicTrainer(model, dataset, verbose=True)

# train the model
top_words, train_theta = trainer.train()

100%|██████████| 200/200 [00:18<00:00, 10.61it/s]


Topic 0: problem know problems tell stuff thing manager widget experience bbs computing switch advice learn appreciate
Topic 1: question believe yes think agree reason say whether fact evidence true know nothing answer anything
Topic 2: writes usa read wrote reading quote write numbers warning writing express compared loaded hole arizona
Topic 3: thanks drive card scsi hard disk memory work speed ibm fast bus computer drivers need
Topic 4: good think know sure opinions get really pretty guess look got maybe run lot anybody
Topic 5: david mark system james gordon systems per corporation jon roger stephen ray total null machine
Topic 6: lines article computer based line internet phone california current entry communications systems previous model access
Topic 7: distribution center brian chris illinois services appreciated deal doug eric care ron greg handle douglas
Topic 8: organization science research technology institute engineering interested canada scientific nec carnegie radar kno

In [8]:
########################### Evaluate ####################################
# get theta (doc-topic distributions)
train_theta, test_theta = trainer.export_theta()

# evaluate topic coherence
# refer to https://github.com/BobXWu/ECRTM

# evaluate topic diversity
TD = topmost.evaluations.compute_topic_diversity(top_words)
print(f"TD: {TD:.5f}")

# evaluate clustering
results = topmost.evaluations.evaluate_clustering(test_theta, dataset.test_labels)
print(results)

# evaluate classification
results = topmost.evaluations.evaluate_classification(train_theta, test_theta, dataset.train_labels, dataset.test_labels)
print(results)

TD: 0.73067
{'Purity': 0.38382899628252787, 'NMI': 0.3483013957003741}


{'acc': 0.5326606479022836, 'macro-F1': 0.5171831529186143}


In [9]:
########################### test new documents ####################################
import torch
from topmost.preprocessing import Preprocessing

preprocessing = Preprocessing()


new_docs = [
    "This is a new document about space, including words like space, satellite, launch, orbit.",
    "This is a new document about Microsoft Windows, including words like windows, files, dos."
]

parsed_new_docs, new_bow = preprocessing.parse(new_docs, vocab=dataset.vocab)
new_theta = trainer.test(torch.as_tensor(new_bow, device=device).float())

print(new_theta.argmax(1))

parse texts: 100%|██████████| 2/2 [00:00<00:00, 20068.44it/s]

[43 16]
